This will be the final notebook in the modeling and expermentation phase, after this i'll have the final model which i can use for the final pipeline and further in testing and deployment 

# Importing Libraries

In [1]:
import pandas as pd
import optuna # For hyperparameter optimization
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import json

# Load the Engineered data

In [2]:
# Define file paths
ENGINEERED_DATA_PATH = '../data/processed/application_train_engineered.csv'
TOP_FEATURES_PATH = '../config/top_features.json'

# Load the engineered data
app_eng_df = pd.read_csv(ENGINEERED_DATA_PATH)
with open(TOP_FEATURES_PATH, 'r') as f:
    top_features = json.load(f)

In [3]:
app_eng_df.head()

,TARGET,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,EMERGENCYSTATE_MODE_Yes,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,PAYMENT_RATE,EXT_SOURCE_PRODUCT,INCOME_QUANTILE_BINS,CREDIT_QUANTILE_BINS,ANNUITY_QUANTILE_BINS,GOODS_PRICE_QUANTILE_BINS,SK_ID_CURR
0,1,0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,False,2.007889,0.121978,0.060749,0.036649,3,1,2,1,100002
1,0,0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,False,4.790750,0.132217,0.027598,0.333073,4,4,3,4,100003
2,0,1,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,False,2.000000,0.100000,0.050000,0.405575,0,0,0,0,100004
3,0,0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,False,2.316167,0.219900,0.094941,0.348166,1,1,3,1,100006
4,0,0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,False,4.222222,0.179963,0.042623,0.172754,1,2,1,2,100007


In [4]:
print(len(top_features), "features loaded.")

60 features loaded.


In [5]:
print("Data loaded successfully. Shape:", app_eng_df.shape)

Data loaded successfully. Shape: (307511, 114)


In [6]:
# Clean column names to match those in top_features
app_eng_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in app_eng_df.columns]

# Preparing Data

In [7]:
# select features and target
X = app_eng_df[top_features]
y = app_eng_df['TARGET']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (246008, 60)
Test set shape: (61503, 60)


# Hyperparameter Tuning 

For this i'll be using the Optuna framework. It's highly efficient and easy to use, much better than grid search or random search. Optuna quickly focuses on the most promising hyperparameter regions and spends less time on bad ones. It also has a pruning feature, which allows it to stop unpromising trials early, saving a huge amount of time.

In [8]:
# Define Optuna objective function
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'n_estinimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 20, 400),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': 42,
        'n_jobs': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train, 
            eval_set=[(X_test, y_test)], 
            eval_metric='auc', 
            callbacks=[lgb.early_stopping(50, verbose=False)]
            )
    preds = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)
    return auc

# Run Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-08-24 15:30:40,366] A new study created in memory with name: no-name-149ca9e5-90bb-4d4a-be1e-1d910432ba43


[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] feature_fraction is set=0.7675761951338131, colsample_bytree=0.4705199351934558 will be ignored. Current value: feature_fraction=0.7675761951338131
[LightGBM] [Warning] bagging_fraction is set=0.6583515016501184, subsample=0.8702235379870574 will be ignored. Current value: bagging_fraction=0.6583515016501184
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] feature_fraction is set=0.7675761951338131, colsample_bytree=0.4705199351934558 will be ignored. Current value: feature_fraction=0.7675761951338131
[LightGBM] [Warning] bagging_fraction 

[I 2025-08-24 15:30:50,980] Trial 0 finished with value: 0.7559630562617083 and parameters: {'n_estimators': 736, 'learning_rate': 0.06788429490190807, 'max_depth': 11, 'num_leaves': 337, 'min_data_in_leaf': 45, 'feature_fraction': 0.7675761951338131, 'bagging_fraction': 0.6583515016501184, 'bagging_freq': 2, 'subsample': 0.8702235379870574, 'colsample_bytree': 0.4705199351934558, 'reg_alpha': 0.0014600003671825835, 'reg_lambda': 0.029873369877351477}. Best is trial 0 with value: 0.7559630562617083.


[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] feature_fraction is set=0.7331405160021978, colsample_bytree=0.7703205665103576 will be ignored. Current value: feature_fraction=0.7331405160021978
[LightGBM] [Warning] bagging_fraction is set=0.5312820816945389, subsample=0.7736852353814642 will be ignored. Current value: bagging_fraction=0.5312820816945389
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] feature_fraction is set=0.7331405160021978, colsample_bytree=0.7703205665103576 will be ignored. Current value: feature_fraction=0.7331405160021978
[LightGBM] [Warning] bagging_fraction 

[I 2025-08-24 15:30:54,359] Trial 1 finished with value: 0.7606598447792441 and parameters: {'n_estimators': 1623, 'learning_rate': 0.14769977135435663, 'max_depth': 4, 'num_leaves': 218, 'min_data_in_leaf': 49, 'feature_fraction': 0.7331405160021978, 'bagging_fraction': 0.5312820816945389, 'bagging_freq': 7, 'subsample': 0.7736852353814642, 'colsample_bytree': 0.7703205665103576, 'reg_alpha': 0.018583052904241757, 'reg_lambda': 0.018640735723171747}. Best is trial 1 with value: 0.7606598447792441.


...

[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=42, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=42
[LightGBM] [Warning] feature_fraction is set=0.8364234493123177, colsample_bytree=0.6452850134643302 will be ignored. Current value: feature_fraction=0.8364234493123177
[LightGBM] [Warning] bagging_fraction is set=0.9623317253392998, subsample=0.6795775368300405 will be ignored. Current value: bagging_fraction=0.9623317253392998
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] feature_fraction is set=0.9825540059673618, colsample_bytree=0.7327063259249915 will be ignored. Current value: feature_fraction=0.9825540059673618
[LightGBM] [Warning] bagging_fraction 

[I 2025-08-24 15:32:57,071] Trial 48 finished with value: 0.7604923345230615 and parameters: {'n_estimators': 816, 'learning_rate': 0.18723791753372176, 'max_depth': 15, 'num_leaves': 55, 'min_data_in_leaf': 86, 'feature_fraction': 0.9825540059673618, 'bagging_fraction': 0.8672318424233271, 'bagging_freq': 3, 'subsample': 0.7624538042450668, 'colsample_bytree': 0.7327063259249915, 'reg_alpha': 0.48467624807483606, 'reg_lambda': 0.1302283837878539}. Best is trial 41 with value: 0.7652571573835126.


[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] feature_fraction is set=0.9825540059673618, colsample_bytree=0.7327063259249915 will be ignored. Current value: feature_fraction=0.9825540059673618
[LightGBM] [Warning] bagging_fraction is set=0.8672318424233271, subsample=0.7624538042450668 will be ignored. Current value: bagging_fraction=0.8672318424233271
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=66, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=66
[LightGBM] [Warning] feature_fraction is set=0.9152752214656884, colsample_bytree=0.5892699602343289 will be ignored. Current value: feature_fraction=0.9152752214656884
[LightGBM] [Warning] bagging_fraction 

[I 2025-08-24 15:33:01,748] Trial 49 finished with value: 0.7577791578439861 and parameters: {'n_estimators': 1036, 'learning_rate': 0.0365749195890617, 'max_depth': 13, 'num_leaves': 128, 'min_data_in_leaf': 66, 'feature_fraction': 0.9152752214656884, 'bagging_fraction': 0.9467071562705948, 'bagging_freq': 4, 'subsample': 0.7293957576920673, 'colsample_bytree': 0.5892699602343289, 'reg_alpha': 0.8827072185155921, 'reg_lambda': 0.06622541297166086}. Best is trial 41 with value: 0.7652571573835126.


[LightGBM] [Warning] Unknown parameter: n_estinimators
[LightGBM] [Warning] min_data_in_leaf is set=66, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=66
[LightGBM] [Warning] feature_fraction is set=0.9152752214656884, colsample_bytree=0.5892699602343289 will be ignored. Current value: feature_fraction=0.9152752214656884
[LightGBM] [Warning] bagging_fraction is set=0.9467071562705948, subsample=0.7293957576920673 will be ignored. Current value: bagging_fraction=0.9467071562705948
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


In [9]:
# Print best parameters
print("Best trial:")
trial = study.best_trial
print(f"  AUC: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  AUC: 0.7652571573835126
  Params: 
    n_estimators: 766
    learning_rate: 0.14661696258280738
    max_depth: 15
    num_leaves: 24
    min_data_in_leaf: 51
    feature_fraction: 0.9287838567079353
    bagging_fraction: 0.9495916039524935
    bagging_freq: 4
    subsample: 0.7231683030135212
    colsample_bytree: 0.6204910468966424
    reg_alpha: 1.1168937615210697
    reg_lambda: 0.10128872382319574


In [10]:
# Save best parameters to a JSON file
best_params = study.best_params

with open('../config/best_lgbm_params.json', 'w') as f:
    json.dump(best_params, f, indent=4)

print("Best hyperparameters saved successfully.")

Best hyperparameters saved successfully.
